# Previsão de Sucesso de Startups

## Objetivo
Este notebook implementa um modelo preditivo para identificar startups com maior probabilidade de sucesso no mercado, utilizando técnicas de machine learning conforme as regras da competição Kaggle.

## Metodologia
- **Modelo**: Random Forest Classifier
- **Otimização**: Grid Search com validação cruzada (5-fold StratifiedKFold)
- **Estratégia**: Treinamento com dataset completo (sem split train/test)
- **Meta**: Acurácia > 80%

## 1. Importação de Bibliotecas

Utilizando apenas as bibliotecas permitidas pela competição: NumPy, Pandas e Scikit-Learn.

In [1]:
# Importação de bibliotecas essenciais
import numpy as np
import pandas as pd
from typing import Tuple

# Scikit-learn: pré-processamento
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Scikit-learn: modelo e otimização
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

# Scikit-learn: métricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Configurações
import warnings
warnings.filterwarnings('ignore')

# Seed para reprodutibilidade
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Bibliotecas importadas com sucesso!")

✓ Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados

Carregando os conjuntos de treino e teste fornecidos pela competição.

In [2]:
# Carregar datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Dataset de Treino: {train_df.shape[0]} linhas x {train_df.shape[1]} colunas")
print(f"Dataset de Teste: {test_df.shape[0]} linhas x {test_df.shape[1]} colunas")

# Visualizar primeiras linhas
print("\n📊 Primeiras linhas do dataset de treino:")
display(train_df.head())

# Informações sobre as colunas
print("\n📋 Informações do dataset:")
train_df.info()

Dataset de Treino: 646 linhas x 33 colunas
Dataset de Teste: 277 linhas x 32 colunas

📊 Primeiras linhas do dataset de treino:


,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
0,719,10.42,13.09,8.98,12.72,4,3,4087500,3,1,...,0,0,1,1,0,0,0,0,1.0,0
1,429,3.79,3.79,NaN,NaN,21,1,45000000,0,0,...,0,0,0,0,0,1,0,0,1.0,1
2,178,0.71,2.28,1.95,2.28,5,2,5200000,2,1,...,0,1,1,0,1,0,0,0,1.0,0
3,197,3.00,5.00,9.62,10.39,16,2,14500000,2,0,...,0,0,0,1,0,1,0,0,2.0,1
4,444,0.66,5.88,6.21,8.61,29,5,70000000,4,1,...,0,0,0,0,1,1,1,1,2.8,1



📋 Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate             646 non-null

## 3. Análise Exploratória Básica

Verificando a distribuição da variável alvo e valores ausentes.

In [3]:
# Distribuição da variável alvo
print("📊 Distribuição da Variável Alvo (labels):")
print(train_df['labels'].value_counts())
print(f"\nPercentual de Sucesso: {train_df['labels'].mean()*100:.2f}%")
print(f"Percentual de Insucesso: {(1-train_df['labels'].mean())*100:.2f}%")

# Análise de valores ausentes
print("\n🔍 Análise de Valores Ausentes:")
missing_data = train_df.isnull().sum()
missing_percent = (missing_data / len(train_df)) * 100
missing_df = pd.DataFrame({
    'Valores Ausentes': missing_data,
    'Percentual': missing_percent
})
missing_df = missing_df[missing_df['Valores Ausentes'] > 0].sort_values('Valores Ausentes', ascending=False)
print(missing_df)

📊 Distribuição da Variável Alvo (labels):
labels
1    418
0    228
Name: count, dtype: int64

Percentual de Sucesso: 64.71%
Percentual de Insucesso: 35.29%

🔍 Análise de Valores Ausentes:
                          Valores Ausentes  Percentual
age_first_milestone_year               138   21.362229
age_last_milestone_year                111   17.182663
age_first_funding_year                  35    5.417957
age_last_funding_year                    9    1.393189


## 3.1. Formulação de Hipóteses de Pesquisa

Para orientar a análise exploratória e fundamentar a construção do modelo preditivo, estabelecemos três hipóteses principais baseadas no domínio de negócio de startups:

### H1 - Capital como Fator Determinante de Sucesso

**Hipótese**: Startups com maior volume de investimento captado e maior número de rodadas de financiamento apresentam probabilidade significativamente superior de alcançar sucesso.

**Variáveis-chave**: `funding_total_usd`, `funding_rounds`, `avg_participants`

**Justificativa**: O capital é fundamental para escala, desenvolvimento de produto e expansão de mercado. Múltiplas rodadas indicam validação contínua por investidores especializados, sugerindo confiança do mercado na proposta de valor da startup.

### H2 - Localização Geográfica e Ecossistema

**Hipótese**: Startups localizadas em hubs tecnológicos consolidados (Califórnia, Nova York, Massachusetts) apresentam maior taxa de sucesso devido ao acesso a talentos, investidores e networking.

**Variáveis-chave**: `is_CA`, `is_NY`, `is_MA`, `is_otherstate`

**Justificativa**: Clusters de inovação oferecem densidade de recursos, capital humano especializado e ecossistema maduro de apoio empresarial. A proximidade com investidores-anjo, VCs e outras startups cria um ambiente propício ao crescimento.

### H3 - Maturidade Operacional e Capacidade de Execução

**Hipótese**: Startups que demonstram capacidade de execução através de relacionamentos estratégicos e marcos (milestones) alcançados apresentam maior probabilidade de sucesso sustentável.

**Variáveis-chave**: `relationships`, `milestones`, `age_first_milestone_year`, `age_last_milestone_year`

**Justificativa**: Relacionamentos indicam networking e parcerias estratégicas. Marcos demonstram capacidade de entrega e progressão estruturada do negócio, além de validação de mercado em diferentes etapas de desenvolvimento.

---

**Metodologia de Validação**: Cada hipótese será testada através de:
1. Análise estatística descritiva comparando startups de sucesso vs. insucesso
2. Visualizações comparativas das distribuições
3. Validação pela importância das features nos modelos preditivos (Random Forest feature importance)

In [4]:
# Validação Exploratória das Hipóteses

print("=" * 70)
print("VALIDAÇÃO EXPLORATÓRIA DAS HIPÓTESES")
print("=" * 70)

# Criar subsets de sucesso e insucesso
success_df = train_df[train_df['labels'] == 1]
failure_df = train_df[train_df['labels'] == 0]

print(f"\n📊 Tamanho das amostras:")
print(f"  - Startups de Sucesso: {len(success_df)}")
print(f"  - Startups de Insucesso: {len(failure_df)}")

# H1 - Análise de Capital
print("\n" + "=" * 70)
print("H1 - CAPITAL COMO FATOR DETERMINANTE")
print("=" * 70)

print("\n💰 Funding Total (USD):")
print(f"  Sucesso - Média: ${success_df['funding_total_usd'].mean():,.0f}")
print(f"  Insucesso - Média: ${failure_df['funding_total_usd'].mean():,.0f}")
print(f"  Diferença: {(success_df['funding_total_usd'].mean() / failure_df['funding_total_usd'].mean() - 1) * 100:.1f}%")

print("\n🔄 Rodadas de Financiamento:")
print(f"  Sucesso - Média: {success_df['funding_rounds'].mean():.2f}")
print(f"  Insucesso - Média: {failure_df['funding_rounds'].mean():.2f}")

print("\n👥 Participantes Médios por Rodada:")
print(f"  Sucesso - Média: {success_df['avg_participants'].mean():.2f}")
print(f"  Insucesso - Média: {failure_df['avg_participants'].mean():.2f}")

# H2 - Análise de Localização
print("\n" + "=" * 70)
print("H2 - LOCALIZAÇÃO GEOGRÁFICA E ECOSSISTEMA")
print("=" * 70)

print("\n🌎 Distribuição por Localização (% de sucesso):")
for location in ['is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate']:
    if location in train_df.columns:
        success_rate = train_df[train_df[location] == 1]['labels'].mean() * 100
        total_count = train_df[location].sum()
        location_name = location.replace('is_', '')
        print(f"  {location_name}: {success_rate:.1f}% (n={total_count})")

# H3 - Análise de Maturidade Operacional
print("\n" + "=" * 70)
print("H3 - MATURIDADE OPERACIONAL E EXECUÇÃO")
print("=" * 70)

print("\n🤝 Relacionamentos Estratégicos:")
print(f"  Sucesso - Média: {success_df['relationships'].mean():.2f}")
print(f"  Insucesso - Média: {failure_df['relationships'].mean():.2f}")

print("\n🎯 Marcos Alcançados:")
print(f"  Sucesso - Média: {success_df['milestones'].mean():.2f}")
print(f"  Insucesso - Média: {failure_df['milestones'].mean():.2f}")

print("\n⏱️  Tempo até Primeiro Milestone (anos):")
print(f"  Sucesso - Média: {success_df['age_first_milestone_year'].mean():.2f}")
print(f"  Insucesso - Média: {failure_df['age_first_milestone_year'].mean():.2f}")

print("\n" + "=" * 70)
print("CONCLUSÕES PRELIMINARES")
print("=" * 70)
print("\nAs análises descritivas fornecem evidências iniciais para validação")
print("das hipóteses. A importância final será confirmada pelo modelo preditivo.")
print("=" * 70)

VALIDAÇÃO EXPLORATÓRIA DAS HIPÓTESES

📊 Tamanho das amostras:
  - Startups de Sucesso: 418
  - Startups de Insucesso: 228

H1 - CAPITAL COMO FATOR DETERMINANTE

💰 Funding Total (USD):
  Sucesso - Média: $36,786,578
  Insucesso - Média: $16,130,875
  Diferença: 128.1%

🔄 Rodadas de Financiamento:
  Sucesso - Média: 2.56
  Insucesso - Média: 1.96

👥 Participantes Médios por Rodada:
  Sucesso - Média: 3.12
  Insucesso - Média: 2.35

H2 - LOCALIZAÇÃO GEOGRÁFICA E ECOSSISTEMA

🌎 Distribuição por Localização (% de sucesso):
  CA: 69.1% (n=353)
  NY: 70.4% (n=71)
  MA: 82.0% (n=61)
  TX: 45.8% (n=24)
  otherstate: 46.3% (n=136)

H3 - MATURIDADE OPERACIONAL E EXECUÇÃO

🤝 Relacionamentos Estratégicos:
  Sucesso - Média: 9.82
  Insucesso - Média: 4.52

🎯 Marcos Alcançados:
  Sucesso - Média: 2.24
  Insucesso - Média: 1.31

⏱️  Tempo até Primeiro Milestone (anos):
  Sucesso - Média: 3.64
  Insucesso - Média: 2.64

CONCLUSÕES PRELIMINARES

As análises descritivas fornecem evidências iniciais para 

## 4. Preparação dos Dados

Separando features e target, removendo ID para modelagem.

In [5]:
# Separar features e target
target = train_df['labels']
features = train_df.drop(columns=['labels'])

# Guardar IDs do teste para submissão
test_ids = test_df['id']

# Remover coluna ID dos features
if 'id' in features.columns:
    features = features.drop(columns=['id'])
if 'id' in test_df.columns:
    test_features = test_df.drop(columns=['id'])
else:
    test_features = test_df.copy()

print(f"✓ Features de treino: {features.shape}")
print(f"✓ Target: {target.shape}")
print(f"✓ Features de teste: {test_features.shape}")

✓ Features de treino: (646, 31)
✓ Target: (646,)
✓ Features de teste: (277, 31)


## 5. Engenharia de Features

Criando features derivadas para melhorar o poder preditivo do modelo:
- **funding_total_log**: Transformação logarítmica do funding total
- **funding_per_round**: Média de captação por rodada
- **funding_age_span**: Tempo entre primeiro e último funding
- **milestone_age_span**: Tempo entre primeiro e último milestone
- **milestones_per_round**: Proporção de milestones por rodada de funding
- **has_funding_history**: Indicador se possui histórico de funding
- **has_milestone_history**: Indicador se possui histórico de milestones

In [6]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria features derivadas que auxiliam modelos baseados em árvore.
    
    Args:
        df: DataFrame com features originais
        
    Returns:
        DataFrame com features originais + features engenheiradas
    """
    result = df.copy()
    
    # Feature 1: Transformação logarítmica do funding total
    if 'funding_total_usd' in result.columns:
        result['funding_total_log'] = np.log1p(result['funding_total_usd'])
    
    # Feature 2: Funding médio por rodada
    if {'funding_total_usd', 'funding_rounds'}.issubset(result.columns):
        result['funding_per_round'] = result['funding_total_usd'] / (
            result['funding_rounds'].replace(0, np.nan)
        )
    
    # Feature 3: Duração do histórico de funding
    if {'age_last_funding_year', 'age_first_funding_year'}.issubset(result.columns):
        result['funding_age_span'] = (
            result['age_last_funding_year'] - result['age_first_funding_year']
        )
    
    # Feature 4: Duração do histórico de milestones
    if {'age_last_milestone_year', 'age_first_milestone_year'}.issubset(result.columns):
        result['milestone_age_span'] = (
            result['age_last_milestone_year'] - result['age_first_milestone_year']
        )
    
    # Feature 5: Proporção milestones por rodada
    if {'milestones', 'funding_rounds'}.issubset(result.columns):
        result['milestones_per_round'] = result['milestones'] / (
            result['funding_rounds'].replace(0, np.nan)
        )
    
    # Feature 6: Indicador de histórico de funding
    if 'age_first_funding_year' in result.columns:
        result['has_funding_history'] = result['age_first_funding_year'].notna().astype(int)
    
    # Feature 7: Indicador de histórico de milestones
    if 'age_first_milestone_year' in result.columns:
        result['has_milestone_history'] = result['age_first_milestone_year'].notna().astype(int)
    
    return result

# Aplicar engenharia de features
features_engineered = engineer_features(features)
test_features_engineered = engineer_features(test_features)

print(f"✓ Features após engenharia: {features_engineered.shape}")
print(f"✓ Novas features criadas: {features_engineered.shape[1] - features.shape[1]}")
print(f"\n📋 Novas features adicionadas:")
new_features = set(features_engineered.columns) - set(features.columns)
for feat in sorted(new_features):
    print(f"  - {feat}")

✓ Features após engenharia: (646, 38)
✓ Novas features criadas: 7

📋 Novas features adicionadas:
  - funding_age_span
  - funding_per_round
  - funding_total_log
  - has_funding_history
  - has_milestone_history
  - milestone_age_span
  - milestones_per_round


## 6. Identificação de Tipos de Features

Classificando features em numéricas e categóricas para aplicar pré-processamento adequado.

In [7]:
# Identificar features numéricas e categóricas
numeric_features = features_engineered.select_dtypes(include=['number', 'bool']).columns.tolist()
categorical_features = features_engineered.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"📊 Features Numéricas: {len(numeric_features)}")
print(f"📊 Features Categóricas: {len(categorical_features)}")
print(f"\nFeatures Categóricas: {categorical_features}")

📊 Features Numéricas: 37
📊 Features Categóricas: 1

Features Categóricas: ['category_code']


## 7. Pipeline de Pré-processamento

Criando pipeline para:
- **Features Numéricas**: Imputação com mediana
- **Features Categóricas**: Imputação com moda + One-Hot Encoding

In [8]:
# Pipeline para features numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))  # Preencher valores ausentes com mediana
])

# Pipeline para features categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Preencher com moda
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-Hot Encoding
])

# Combinar transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print("✓ Pipeline de pré-processamento configurado!")

✓ Pipeline de pré-processamento configurado!


## 8. Configuração do Modelo Random Forest

Utilizando Random Forest com Grid Search para encontrar os melhores hiperparâmetros.

### Hiperparâmetros testados:
- **n_estimators**: Número de árvores na floresta
- **max_depth**: Profundidade máxima das árvores
- **min_samples_split**: Mínimo de amostras para split
- **min_samples_leaf**: Mínimo de amostras por folha
- **max_features**: Número de features por split
- **class_weight**: Balanceamento de classes

In [9]:
# Modelo base: Random Forest Classifier
rf_model = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1  # Usar todos os cores disponíveis
)

# Pipeline completo: pré-processamento + modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_model)
])

# Grid de hiperparâmetros para otimização
param_grid = {
    'model__n_estimators': [300, 500, 800],
    'model__max_depth': [None, 16, 24, 32],
    'model__min_samples_split': [2, 4],
    'model__min_samples_leaf': [1, 2],
    'model__max_features': ['sqrt', 'log2', 0.7],
    'model__class_weight': [None, 'balanced_subsample']
}

print("✓ Modelo configurado!")
print(f"\n📊 Total de combinações a testar: {np.prod([len(v) for v in param_grid.values()])}")

✓ Modelo configurado!

📊 Total de combinações a testar: 288


## 9. Grid Search com Validação Cruzada (5-Fold)

Conforme regras da competição:
- Usando **StratifiedKFold** com 5 folds
- Treinando no dataset completo (sem split train/test)
- Otimizando para **acurácia**

In [ ]:
# Configurar validação cruzada estratificada
cv_strategy = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=RANDOM_STATE
)

# Configurar Grid Search
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

print("✓ Grid Search configurado!")
print("\n🚀 Iniciando treinamento e otimização...")
print("⏳ Este processo pode levar alguns minutos...\n")

# Treinar modelo
grid_search.fit(features_engineered, target)

print("\n✅ Treinamento concluído!")

✓ Grid Search configurado!

🚀 Iniciando treinamento e otimização...
⏳ Este processo pode levar alguns minutos...

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


## 10. Resultados da Otimização

Analisando os melhores hiperparâmetros encontrados e a performance do modelo.

In [ ]:
# Melhores hiperparâmetros
print("🏆 Melhores Hiperparâmetros Encontrados:")
print("="*60)
for param, value in grid_search.best_params_.items():
    print(f"  {param.replace('model__', '')}: {value}")

# Melhor score (acurácia média na validação cruzada)
best_cv_score = grid_search.best_score_
print(f"\n📊 Acurácia Média (5-Fold CV): {best_cv_score:.4f} ({best_cv_score*100:.2f}%)")

# Verificar se atingimos a meta de 80%
if best_cv_score >= 0.80:
    print("\n✅ META ATINGIDA! Acurácia > 80%")
else:
    print(f"\n⚠️  Acurácia atual: {best_cv_score*100:.2f}% (meta: 80%)")

# Análise dos top 10 modelos
print("\n📈 Top 10 Combinações de Hiperparâmetros:")
results_df = pd.DataFrame(grid_search.cv_results_)
top_10 = results_df.nlargest(10, 'mean_test_score')[[
    'mean_test_score', 'std_test_score', 'rank_test_score'
]]
top_10['mean_test_score'] = top_10['mean_test_score'].apply(lambda x: f"{x:.4f}")
top_10['std_test_score'] = top_10['std_test_score'].apply(lambda x: f"{x:.4f}")
print(top_10.to_string(index=False))

🏆 Melhores Hiperparâmetros Encontrados:
  class_weight: None
  max_depth: None
  max_features: sqrt
  min_samples_leaf: 2
  min_samples_split: 2
  n_estimators: 300

📊 Acurácia Média (5-Fold CV): 0.7894 (78.94%)

⚠️  Acurácia atual: 78.94% (meta: 80%)

📈 Top 10 Combinações de Hiperparâmetros:
mean_test_score std_test_score  rank_test_score
         0.7894         0.0202                1
         0.7894         0.0286                1
         0.7894         0.0202                1
         0.7894         0.0286                1
         0.7894         0.0202                1
         0.7894         0.0286                1
         0.7894         0.0202                1
         0.7894         0.0286                1
         0.7894         0.0202                1
         0.7894         0.0286                1


## 11. Avaliação do Modelo com Validação Cruzada

Calculando métricas detalhadas: Acurácia, Precisão, Recall e F1-Score.

In [ ]:
# Obter o melhor modelo
best_model = grid_search.best_estimator_

# Calcular métricas com validação cruzada
print("📊 Métricas de Avaliação (Validação Cruzada 5-Fold):")
print("="*60)

# Acurácia
accuracy_scores = cross_val_score(best_model, features_engineered, target, 
                                   cv=cv_strategy, scoring='accuracy')
print(f"Acurácia: {accuracy_scores.mean():.4f} (±{accuracy_scores.std():.4f})")

# Precisão
precision_scores = cross_val_score(best_model, features_engineered, target, 
                                   cv=cv_strategy, scoring='precision')
print(f"Precisão: {precision_scores.mean():.4f} (±{precision_scores.std():.4f})")

# Recall
recall_scores = cross_val_score(best_model, features_engineered, target, 
                                cv=cv_strategy, scoring='recall')
print(f"Recall:   {recall_scores.mean():.4f} (±{recall_scores.std():.4f})")

# F1-Score
f1_scores = cross_val_score(best_model, features_engineered, target, 
                            cv=cv_strategy, scoring='f1')
print(f"F1-Score: {f1_scores.mean():.4f} (±{f1_scores.std():.4f})")

print("\n📌 Nota: Valores ± representam o desvio padrão entre os folds")

📊 Métricas de Avaliação (Validação Cruzada 5-Fold):
Acurácia: 0.7894 (±0.0202)
Acurácia: 0.7894 (±0.0202)
Precisão: 0.7894 (±0.0182)
Precisão: 0.7894 (±0.0182)
Recall:   0.9209 (±0.0271)
Recall:   0.9209 (±0.0271)
F1-Score: 0.8498 (±0.0146)

📌 Nota: Valores ± representam o desvio padrão entre os folds
F1-Score: 0.8498 (±0.0146)

📌 Nota: Valores ± representam o desvio padrão entre os folds


## 12. Importância das Features

Analisando quais features mais contribuem para as previsões do modelo.

In [ ]:
# Obter importância das features
try:
    # Acessar o modelo Random Forest dentro do pipeline
    rf_classifier = best_model.named_steps['model']
    
    # Obter nomes das features após pré-processamento
    preprocessor_fitted = best_model.named_steps['preprocessor']
    
    # Features numéricas mantêm seus nomes
    numeric_feature_names = numeric_features
    
    # Features categóricas são expandidas pelo OneHotEncoder
    categorical_feature_names = []
    if len(categorical_features) > 0:
        ohe = preprocessor_fitted.named_transformers_['cat'].named_steps['onehot']
        categorical_feature_names = ohe.get_feature_names_out(categorical_features).tolist()
    
    # Combinar nomes
    all_feature_names = numeric_feature_names + categorical_feature_names
    
    # Criar DataFrame com importâncias
    feature_importance_df = pd.DataFrame({
        'Feature': all_feature_names,
        'Importância': rf_classifier.feature_importances_
    }).sort_values('Importância', ascending=False)
    
    print("🌟 Top 20 Features Mais Importantes:")
    print("="*60)
    print(feature_importance_df.head(20).to_string(index=False))
    
except Exception as e:
    print(f"⚠️  Não foi possível extrair importância das features: {e}")

🌟 Top 20 Features Mais Importantes:
                 Feature  Importância
           relationships     0.111812
 age_last_milestone_year     0.078475
       funding_total_log     0.076583
       funding_total_usd     0.072489
   age_last_funding_year     0.064415
       funding_per_round     0.063645
  age_first_funding_year     0.058331
age_first_milestone_year     0.053964
        avg_participants     0.051869
    milestones_per_round     0.046765
              milestones     0.044608
      milestone_age_span     0.043821
        funding_age_span     0.040531
          funding_rounds     0.024481
   has_milestone_history     0.019680
           is_otherstate     0.011797
              has_roundB     0.011140
                  has_VC     0.009340
              has_roundA     0.009064
              has_roundC     0.008686


## 13. Treinamento Final e Predições

Gerando predições para o conjunto de teste e criando o arquivo de submissão.

In [ ]:
# O modelo já está treinado com todos os dados (grid_search.fit)
# Agora vamos fazer as predições no conjunto de teste

print("🔮 Gerando predições para o conjunto de teste...")
predictions = best_model.predict(test_features_engineered)

# Análise das predições
print(f"\n📊 Distribuição das Predições:")
print(f"  Sucesso (1): {(predictions == 1).sum()} ({(predictions == 1).sum()/len(predictions)*100:.2f}%)")
print(f"  Insucesso (0): {(predictions == 0).sum()} ({(predictions == 0).sum()/len(predictions)*100:.2f}%)")

print("\n✓ Predições concluídas!")

🔮 Gerando predições para o conjunto de teste...

📊 Distribuição das Predições:
  Sucesso (1): 205 (74.01%)
  Insucesso (0): 72 (25.99%)

✓ Predições concluídas!


## 14. Geração do Arquivo de Submissão

Criando o arquivo CSV no formato exigido pela competição Kaggle.

In [ ]:
# Criar DataFrame de submissão
submission_df = pd.DataFrame({
    'id': test_ids,
    'labels': predictions
})

# Gerar nome de arquivo incremental para submissão
import os
import re

def get_next_submission_filename(base_name="submission", ext="csv"):
    files = [f for f in os.listdir() if re.match(rf"{base_name}(_v\\d+)?\\.{ext}$", f)]
    if not files:
        return f"{base_name}.{ext}"
    versions = [0]  # 0 para submission.csv
    for f in files:
        m = re.search(r'_v(\\d+)', f)
        if m:
            versions.append(int(m.group(1)))
    next_version = max(versions) + 1
    if f"{base_name}.{ext}" in files and next_version == 1:
        # submission.csv já existe, então próxima é _v2
        next_version = 2
    if next_version == 1:
        return f"{base_name}.{ext}"
    else:
        return f"{base_name}_v{next_version}.{ext}"

submission_filename = get_next_submission_filename()
submission_df.to_csv(submission_filename, index=False)

print(f"✅ Arquivo de submissão criado: {submission_filename}")
print(f"\n📋 Preview do arquivo de submissão:")
display(submission_df.head(10))
print(f"\nTotal de predições: {len(submission_df)}")


✅ Arquivo de submissão criado: submission.csv

📋 Preview do arquivo de submissão:


,id,labels
0,70,1
1,23,0
2,389,1
3,872,1
4,920,0
5,690,1
6,588,0
7,144,1
8,875,1
9,900,1



Total de predições: 277


## 15. Resumo Final

Consolidando os resultados e próximos passos.

In [ ]:
print("="*70)
print("📊 RESUMO FINAL DO MODELO")
print("="*70)
print(f"\n🤖 Modelo: Random Forest Classifier")
print(f"🔧 Otimização: Grid Search com {len(param_grid)} hiperparâmetros")
print(f"📊 Validação: StratifiedKFold (5 folds)")
print(f"\n🎯 RESULTADOS:")
print(f"  - Acurácia CV: {best_cv_score:.4f} ({best_cv_score*100:.2f}%)")
print(f"  - Precisão CV: {precision_scores.mean():.4f}")
print(f"  - Recall CV: {recall_scores.mean():.4f}")
print(f"  - F1-Score CV: {f1_scores.mean():.4f}")

if best_cv_score >= 0.80:
    print(f"\n✅ META ALCANÇADA! Acurácia > 80%")
else:
    print(f"\n⚠️  Acurácia atual: {best_cv_score*100:.2f}% (meta: 80%)")
    print("\n💡 Sugestões para melhorar:")
    print("  - Criar mais features engenheiradas")
    print("  - Testar outros ranges de hiperparâmetros")
    print("  - Considerar ensemble com outros modelos")
    print("  - Analisar e tratar outliers")

print(f"\n📁 Arquivo gerado: {submission_filename}")
print(f"\n🚀 Próximos passos:")
print("  1. Submeter o arquivo submission.csv no Kaggle")
print("  2. Verificar a acurácia no leaderboard público")
print("  3. Iterar melhorias se necessário")
print("\n" + "="*70)

📊 RESUMO FINAL DO MODELO

🤖 Modelo: Random Forest Classifier
🔧 Otimização: Grid Search com 6 hiperparâmetros
📊 Validação: StratifiedKFold (5 folds)

🎯 RESULTADOS:
  - Acurácia CV: 0.7894 (78.94%)
  - Precisão CV: 0.7894
  - Recall CV: 0.9209
  - F1-Score CV: 0.8498

⚠️  Acurácia atual: 78.94% (meta: 80%)

💡 Sugestões para melhorar:
  - Criar mais features engenheiradas
  - Testar outros ranges de hiperparâmetros
  - Considerar ensemble com outros modelos
  - Analisar e tratar outliers

📁 Arquivo gerado: submission.csv

🚀 Próximos passos:
  1. Submeter o arquivo submission.csv no Kaggle
  2. Verificar a acurácia no leaderboard público
  3. Iterar melhorias se necessário

